In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import warnings
import os
from sklearn.svm import SVC
warnings.filterwarnings("ignore")

Using TensorFlow backend.


## Dataset Pre-processing

In [93]:
TASKS = ["climbing","walking"]
LABELS = {
    "climbing" : 1,
    "walking" : 2,
}
SENSORS = ["Accelerometer","Gyroscope"]
WINDOW_LENGTH = 100
dataset_path = "dataset_new/train/"

In [94]:
data = {}
for sensors in SENSORS:
    data[sensors] = []
for vals in TASKS:
    sub_dirs = [x for x in os.walk(dataset_path + vals)]
    datapoints = sub_dirs[0][1]
    for pts in datapoints:
        for sensors in SENSORS:
            file_path = dataset_path + vals + "/" + pts + "/" +sensors + ".csv"
            df = pd.read_csv(file_path)
            df["Label"] = LABELS[vals]
            data[sensors].append(df)

In [95]:
acc = data["Accelerometer"]
gyro = data["Gyroscope"]
combine = []

In [96]:
for i in range(len(acc)):
    print(acc[i].shape,gyro[i].shape)

(8776, 6) (9005, 6)
(1808, 6) (1860, 6)
(4907, 6) (5040, 6)


In [97]:
N = 50
for i in range(len(acc)):
    length = min(acc[i].shape[0],gyro[i].shape[0])
    length = length - length%100
    acc[i] = acc[i][:length]
    gyro[i] = gyro[i][:length]
    acc[i] = acc[i][N:-N]
    gyro[i] = gyro[i][N:-N]

In [98]:
for i in range(len(acc)):
    print(acc[i].shape,gyro[i].shape)
    try:
        acc[i].drop(acc[i][["Timestamp","Milliseconds"]],inplace=True,axis=1)
        gyro[i].drop(gyro[i][["Timestamp","Milliseconds"]],inplace=True,axis=1)
    except:
        pass
    gyro[i] = gyro[i].reset_index(drop=True)
    acc[i] = acc[i].reset_index(drop=True)
    acc[i].columns = ["ACCX","ACCY","ACCZ","OUTPUT"]
    gyro[i].columns = ["GYROX","GYROY","GYROZ","OUTPUT"]
    if(acc[i]['OUTPUT'].equals(gyro[i]["OUTPUT"])):
        acc[i].drop(acc[i][["OUTPUT"]],axis=1,inplace=True)
        df = pd.concat([acc[i], gyro[i]],axis=1)
        combine.append(df)
    else:
        print("Error in merging tables")

(8600, 6) (8600, 6)
(1700, 6) (1700, 6)
(4800, 6) (4800, 6)


In [99]:
for i in range(len(combine)):
    print(combine[i].shape)

(8600, 7)
(1700, 7)
(4800, 7)


In [100]:
x_train = []
y_train = []
x_test = []
y_test = []
for values in combine:
    i=0
    while(i<= 0.7 * values.shape[0]- WINDOW_LENGTH):
        i = int(i)
        temp = values[i:i+WINDOW_LENGTH]
        temp_input = temp.drop(temp[['OUTPUT']],axis=1)
        temp_output = temp[['OUTPUT']]
        x_train.append(temp_input.values)
        y_train.append(temp_output.iloc[0])
        i = i + WINDOW_LENGTH/2
    while(i <= values.shape[0] - WINDOW_LENGTH):
        i = int(i)
        temp = values[i:i+WINDOW_LENGTH]
        temp_input = temp.drop(temp[['OUTPUT']],axis=1)
        temp_output = temp[['OUTPUT']]
        x_test.append(temp_input.values)
        y_test.append(temp_output.iloc[0])
        i = i + WINDOW_LENGTH/2

In [111]:
x_train = np.array(x_train)
x_train_mean = []
x_train_std = []

x_test = np.array(x_test)
x_test_mean = []
x_test_std = []

for layer_one in x_train:
    df = pd.DataFrame(layer_one)
    df_mean = df.rolling(100).mean()
    df_mean.dropna(inplace=True)
    df_mean.reset_index(drop=True, inplace=True)
    add_mean = np.array(df_mean.loc[0])
    x_train_mean.append(add_mean)
    
    df_std = df.rolling(100).std()
    df_std.dropna(inplace=True)
    df_std.reset_index(drop=True, inplace=True)
    add_std = np.array(df_std.loc[0])
    x_train_std.append(add_std)

for layer_one in x_test:
    df = pd.DataFrame(layer_one)
    df_mean = df.rolling(100).mean()
    df_mean.dropna(inplace=True)
    df_mean.reset_index(drop=True, inplace=True)
    add_mean = np.array(df_mean.loc[0])
    x_test_mean.append(add_mean)
    
    df_std = df.rolling(100).std()
    df_std.dropna(inplace=True)
    df_std.reset_index(drop=True, inplace=True)
    add_std = np.array(df_std.loc[0])
    x_test_std.append(add_std)



In [114]:
x_train_mean_df = pd.DataFrame(x_train_mean)
x_train_std_df = pd.DataFrame(x_train_std)
x_train_featured_df = x_train_mean_df.join(x_train_std_df, lsuffix = 'mean', rsuffix = 'std')
x_train_featured = np.array(x_train_featured_df)

x_train_mean_df = pd.DataFrame(x_train_mean)
x_train_std_df = pd.DataFrame(x_train_std)
x_train_featured_df = x_train_mean_df.join(x_train_std_df, lsuffix = 'mean', rsuffix = 'std')
x_train_featured = np.array(x_train_featured_df)
x_train_featured

array([[-1.310215  ,  1.17302764,  9.78272034, ...,  0.52073494,
         0.61953981,  0.36297557],
       [-0.92605317,  0.89971695, 10.12663574, ...,  0.53757168,
         0.61888026,  0.33879463],
       [-0.89474976,  1.03776627, 10.28174408, ...,  0.42617391,
         0.54360879,  0.37191048],
       ...,
       [-0.4182428 ,  1.02815414,  9.98152328, ...,  0.26842086,
         0.36528772,  0.38860891],
       [-0.41635178,  1.2272792 ,  9.89510483, ...,  0.26747761,
         0.32495962,  0.4057471 ],
       [-0.29691712,  1.27813187,  9.82352984, ...,  0.24758335,
         0.30603538,  0.3464894 ]])

In [115]:
y_train = np.array(y_train)

In [116]:
x_train_featured = np.array(x_train_featured)
print(x_train_featured.shape)
print(y_train.shape)

(207, 12)
(207, 1)


In [117]:
x_test_featured = np.array(x_test_featured)
y_test = np.array(y_test)

In [118]:
clf = SVC(gamma='auto')
clf.fit(x_train_featured, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [119]:
clf.score(x_test_featured, y_test)

ValueError: X.shape[1] = 6 should be equal to 12, the number of features at training time